IMPORT LIBRARIES

In [46]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import csv
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.impute import KNNImputer
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs
%matplotlib inline
from scipy.stats import zscore
import datetime as dt

from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor


READING DATA AND JOINS

In [2]:
#READING DATA
path_2016 = "C:\\Users\\rolan\\OneDrive\\Desktop\\GTech OMSA\\CSE-6242\\Group Project\\data\\properties_2016.csv"
path_2017 = "C:\\Users\\rolan\\OneDrive\\Desktop\\GTech OMSA\\CSE-6242\\Group Project\\data\\properties_2017.csv"
trainpath_2016 = "C:\\Users\\rolan\\OneDrive\\Desktop\\GTech OMSA\\CSE-6242\\Group Project\data\\train_2016_v2.csv"
trainpath_2017 = "C:\\Users\\rolan\\OneDrive\\Desktop\\GTech OMSA\\CSE-6242\\Group Project\\data\\train_2017.csv"
df_2016 = pd.read_csv(path_2016, encoding = 'utf-8')
df_train_2016 = pd.read_csv(trainpath_2016, encoding = 'utf-8')
df_2017 = pd.read_csv(path_2017, encoding = 'utf-8')
df_train_2017 = pd.read_csv(trainpath_2017, encoding = 'utf-8')

# path_2016 = 'C:\\Users\\guanh\\OneDrive\\Documents\\OMSA\\CSE6242\\GroupProject\\data\\properties_2016.csv'
# path_2017 = 'C:\\Users\\guanh\\OneDrive\\Documents\\OMSA\\CSE6242\\GroupProject\\data\\properties_2017.csv'
# trainpath_2016 = 'C:\\Users\\guanh\\OneDrive\\Documents\\OMSA\\CSE6242\\GroupProject\\data\\train_2016_v2.csv'
# trainpath_2017 = 'C:\\Users\\guanh\\OneDrive\\Documents\\OMSA\\CSE6242\\GroupProject\\data\\train_2017.csv'

C:\Users\rolan\AppData\Local\Temp\ipykernel_12280\1916595558.py:6: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2016 = pd.read_csv(path_2016, encoding = 'utf-8')
C:\Users\rolan\AppData\Local\Temp\ipykernel_12280\1916595558.py:8: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2017 = pd.read_csv(path_2017, encoding = 'utf-8')


In [3]:
#join the train with feature data
final_16 = df_2016.merge(df_train_2016, on = 'parcelid', how = 'inner')
final_16.head(5)

final_17 = df_2017.merge(df_train_2017, on = 'parcelid', how = 'inner')
final_17.head(5)

# final_16.info()
# final_17.info()

#ensure both 2016 and 2017 data have the same column before we combine and remove dupes
final_16.columns == final_17.columns

#remove duplicates
final = pd.concat([final_16, final_17],ignore_index=True)
print('Number of data points BEFORE removing duplicates::', final.shape)

final = final.drop_duplicates(subset=['parcelid'],keep='last')
print('Number of data points AFTER removing duplicates::', final.shape)


Number of data points BEFORE removing duplicates:: (167888, 60)
Number of data points AFTER removing duplicates:: (165210, 60)


In [4]:
# grab screenshot
final.describe()


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,1.652100e+05,53057.000000,459.000000,92.000000,165177.000000,165177.000000,30.000000,105366.000000,163422.000000,1262.0,...,164182.000000,37610.000000,1.646920e+05,1.651750e+05,165177.000000,1.651740e+05,165166.000000,4529.000000,1.643410e+05,165210.000000
mean,1.299793e+07,1.814897,7.305011,697.336957,2.291121,3.043608,3.966667,6.017444,2.315218,66.0,...,1968.668003,1.438421,1.848479e+05,4.738407e+05,2015.468473,2.895351e+05,5999.531543,13.817841,6.049424e+13,0.012928
std,3.010371e+06,2.971235,2.745897,586.844051,1.000715,1.148108,0.182574,1.884160,0.977816,0.0,...,23.748635,0.544636,2.192159e+05,6.033909e+05,0.499007,4.465328e+05,7214.795709,2.085555,1.062366e+12,0.164396
min,1.071174e+07,1.000000,2.000000,38.000000,0.000000,0.000000,3.000000,1.000000,1.000000,66.0,...,1824.000000,1.000000,4.400000e+01,2.200000e+01,2015.000000,2.200000e+01,19.920000,3.000000,6.037101e+13,-4.655420
25%,1.154960e+07,1.000000,7.000000,278.250000,2.000000,2.000000,4.000000,4.000000,2.000000,66.0,...,1953.000000,1.000000,8.295700e+04,2.036995e+05,2015.000000,8.449100e+04,2809.977500,14.000000,6.037312e+13,-0.025300
50%,1.254324e+07,1.000000,7.000000,594.000000,2.000000,3.000000,4.000000,7.000000,2.000000,66.0,...,1970.000000,1.000000,1.345760e+05,3.509280e+05,2015.000000,1.984640e+05,4512.360000,14.000000,6.037604e+13,0.006000
75%,1.422105e+07,1.000000,7.000000,822.000000,3.000000,4.000000,4.000000,7.000000,3.000000,66.0,...,1987.000000,2.000000,2.148290e+05,5.550110e+05,2016.000000,3.560658e+05,6927.247500,15.000000,6.059042e+13,0.038300
max,1.676893e+08,13.000000,21.000000,3560.000000,20.000000,16.000000,4.000000,12.000000,20.000000,66.0,...,2016.000000,6.000000,1.142179e+07,4.906124e+07,2016.000000,4.895220e+07,586639.300000,99.000000,4.830301e+14,5.262999


In [5]:
#grab screenshot

final.head(10)


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate
0,17073783,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,...,115087.0,191811.0,2015.0,76724.0,2015.06,NaN,NaN,6.111002e+13,0.0953,2016-01-27
2,17100444,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,...,33619.0,47853.0,2015.0,14234.0,591.64,NaN,NaN,6.111001e+13,0.0060,2016-05-27
3,17102429,NaN,NaN,NaN,1.5,2.0,NaN,NaN,1.5,NaN,...,45609.0,62914.0,2015.0,17305.0,682.78,NaN,NaN,6.111001e+13,-0.0566,2016-06-07
4,17109604,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,NaN,...,277000.0,554000.0,2015.0,277000.0,5886.92,NaN,NaN,6.111001e+13,0.0573,2016-08-08
5,17125829,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,66.0,...,222070.0,289609.0,2015.0,67539.0,3110.44,NaN,NaN,6.111005e+13,0.0564,2016-08-26
6,17132911,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,...,185000.0,526000.0,2015.0,341000.0,5632.20,NaN,NaN,6.111006e+13,0.0315,2016-07-08
7,17134926,NaN,NaN,NaN,2.5,5.0,NaN,NaN,2.5,NaN,...,342611.0,571086.0,2015.0,228475.0,6109.94,NaN,NaN,6.111006e+13,0.0257,2016-05-11
8,17139988,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,...,231297.0,462594.0,2015.0,231297.0,5026.40,NaN,NaN,6.111005e+13,0.0020,2016-06-15
9,17167359,NaN,NaN,NaN,1.0,3.0,NaN,NaN,1.0,NaN,...,134251.0,268502.0,2015.0,134251.0,3217.06,NaN,NaN,6.111004e+13,-0.0576,2016-05-27
10,17179760,NaN,NaN,NaN,2.0,4.0,NaN,NaN,2.0,NaN,...,42257.0,61453.0,2015.0,19196.0,702.40,NaN,NaN,6.111005e+13,-0.0131,2016-10-07


In [6]:
#grab screenshot


final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165210 entries, 0 to 167887
Data columns (total 60 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   parcelid                      165210 non-null  int64  
 1   airconditioningtypeid         53057 non-null   float64
 2   architecturalstyletypeid      459 non-null     float64
 3   basementsqft                  92 non-null      float64
 4   bathroomcnt                   165177 non-null  float64
 5   bedroomcnt                    165177 non-null  float64
 6   buildingclasstypeid           30 non-null      float64
 7   buildingqualitytypeid         105366 non-null  float64
 8   calculatedbathnbr             163422 non-null  float64
 9   decktypeid                    1262 non-null    float64
 10  finishedfloor1squarefeet      12734 non-null   float64
 11  calculatedfinishedsquarefeet  164342 non-null  float64
 12  finishedsquarefeet12          157018 non-nul

CONVERT VARIABLES TO RESPECTIVE DATA TYPES

In [7]:
#before EDA, we need to convert the data to the right format

def convertDataType(list, df, type):
    for i in list:
        df[i] = df[i].astype(type)

def convertDate(list, df):
    for i in list:
        df[i] = pd.to_datetime(df[i], infer_datetime_format=True)

#if theres id in a column name, we take it out and cast it as category
cat_var = []
year_var = []

for i in final.columns:
    if 'id' in i:
        cat_var.append(i)

    elif 'year' in i or 'date' in i:
        year_var.append(i)

#print to check if it makes sense
print('CATEGORY COLUMNS: ' ,cat_var)
print('YEAR COLUMNS:  ' ,year_var)

#Add additional category variable manually
# other_cat = ['fips', 'propertycountylandusecode', 'censustractandblock', 'rawcensustractandblock', 'propertyzoningdesc'
#              ]

# cat_var.extend(other_cat)

# cat_var
#run method to convert these fields to category or Timestampes
convertDataType(cat_var, final, 'category')
convertDate(year_var, final)

final.columns

CATEGORY COLUMNS:  ['parcelid', 'airconditioningtypeid', 'architecturalstyletypeid', 'buildingclasstypeid', 'buildingqualitytypeid', 'decktypeid', 'heatingorsystemtypeid', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'propertylandusetypeid', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'storytypeid', 'typeconstructiontypeid']
YEAR COLUMNS:   ['yearbuilt', 'assessmentyear', 'taxdelinquencyyear', 'transactiondate']


Index(['parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildin

FILL MISSING VALUES WITH ZEROES

Before removing high null columns, we want to see which columns can we reasonably fill with 0, such as dummy variables -------

poolcnt has a value of 1 or NaN ---- we will fill Nan with 0

fireplaceflag, hashottuborspa has value of True or NaN ------- we will replace NaN with False

numberofstories has unique values of NaN,1,2....6 -------- we will replace NaN with 0 here

taxdelinquencyflag is either Y or N ---- we will replace with 0



In [8]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165210 entries, 0 to 167887
Data columns (total 60 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   parcelid                      165210 non-null  category      
 1   airconditioningtypeid         53057 non-null   category      
 2   architecturalstyletypeid      459 non-null     category      
 3   basementsqft                  92 non-null      float64       
 4   bathroomcnt                   165177 non-null  float64       
 5   bedroomcnt                    165177 non-null  float64       
 6   buildingclasstypeid           30 non-null      category      
 7   buildingqualitytypeid         105366 non-null  category      
 8   calculatedbathnbr             163422 non-null  float64       
 9   decktypeid                    1262 non-null    category      
 10  finishedfloor1squarefeet      12734 non-null   float64       
 11  calculatedfin

In [9]:
#since this is true false, we will convert to 1 and 0
final['fireplaceflag'] = final['fireplaceflag'].apply(lambda x: 1 if x == 'True' else 0)
final['hashottuborspa'] = final['hashottuborspa'].apply(lambda x: 1 if x == 'True' else 0)
final['taxdelinquencyflag'] = final['taxdelinquencyflag'].apply(lambda x: 1 if x == 'Y' else 0)

In [10]:
#now, lets convert the binary variables to float or int, so that regression can be ran later

to_int = ['poolcnt', 'hashottuborspa', 'numberofstories', 'fireplaceflag', 'taxdelinquencyflag', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7']

convertDataType(to_int, final, float)

final.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 165210 entries, 0 to 167887
Data columns (total 60 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   parcelid                      165210 non-null  category      
 1   airconditioningtypeid         53057 non-null   category      
 2   architecturalstyletypeid      459 non-null     category      
 3   basementsqft                  92 non-null      float64       
 4   bathroomcnt                   165177 non-null  float64       
 5   bedroomcnt                    165177 non-null  float64       
 6   buildingclasstypeid           30 non-null      category      
 7   buildingqualitytypeid         105366 non-null  category      
 8   calculatedbathnbr             163422 non-null  float64       
 9   decktypeid                    1262 non-null    category      
 10  finishedfloor1squarefeet      12734 non-null   float64       
 11  calculatedfin

In [11]:

# #fill all the NaN with 0 here
fillzero = ['poolcnt', 'hashottuborspa', 'numberofstories', 'fireplaceflag', 'taxdelinquencyflag', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7']

# # convertDataType(fillzero,final, 'int')
final[fillzero] = final[fillzero].fillna(0)


DEALING WITH NULL VALUES

In [12]:
#columns that have null values in terms of count and percentages
final.isnull().sum()
null_percent = final.isnull().mean()*100
null_percent

#find out rows where nulls is less than 1% of total, and remove those rows
onepercentnull = null_percent[null_percent< 2].index.tolist()
final = final.dropna(subset = onepercentnull)


# getting high null COLUMNS greater than set threshold %, and remove entire column
threshold = 60
high_nulls = null_percent[null_percent> threshold].index.tolist()
print("Columns with more than x% of missing values:")
for column in high_nulls:
    print("{}: {:.2f}%".format(column, null_percent[column]))


high_nulls

Columns with more than x% of missing values:
airconditioningtypeid: 67.89%
architecturalstyletypeid: 99.72%
basementsqft: 99.94%
buildingclasstypeid: 99.98%
decktypeid: 99.24%
finishedfloor1squarefeet: 92.29%
finishedsquarefeet13: 99.96%
finishedsquarefeet15: 96.09%
finishedsquarefeet50: 92.29%
finishedsquarefeet6: 99.52%
fireplacecnt: 89.30%
garagecarcnt: 66.89%
garagetotalsqft: 66.89%
poolsizesum: 98.90%
regionidneighborhood: 60.15%
storytypeid: 99.94%
threequarterbathnbr: 86.76%
typeconstructiontypeid: 99.69%
yardbuildingsqft17: 96.99%
yardbuildingsqft26: 99.90%
taxdelinquencyyear: 97.26%


['airconditioningtypeid',
 'architecturalstyletypeid',
 'basementsqft',
 'buildingclasstypeid',
 'decktypeid',
 'finishedfloor1squarefeet',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fireplacecnt',
 'garagecarcnt',
 'garagetotalsqft',
 'poolsizesum',
 'regionidneighborhood',
 'storytypeid',
 'threequarterbathnbr',
 'typeconstructiontypeid',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'taxdelinquencyyear']

In [13]:
len(high_nulls) #25 columns

#Drop columns with more than threshold% of missing values

final = final.drop(columns = high_nulls, axis = 1)
len(final.columns)


39

In [14]:
#this code allows the plots to be opened in a new window
%matplotlib tk

CORRELATION MATRIX

Here, we want to remove highly correlated variables

In [15]:
#run correlation matrix on numeric columns
num_cols = final.select_dtypes(include = ['int64', 'float64'])
plt.close('all')
correlation_matrix = num_cols.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.plot([1,2])


In [16]:
#extract pairs that have correlation more than threshold
threshold2 = 0.8

correlation_mask = (correlation_matrix > threshold2) | (correlation_matrix < -threshold2)

strong_correlations = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i + 1, len(correlation_matrix.columns)):
        if correlation_mask.iloc[i, j]:
            strong_correlations.append((correlation_matrix.columns[i], correlation_matrix.columns[j], correlation_matrix.iloc[i, j]))

#export strong correlations to a csv to view
filename = 'strongcorrelation.csv'

with open(filename, 'w', newline = '') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['var1, var2', 'correlation'])
    writer.writerows(strong_correlations)


#check your working directory to get the csv file


In [17]:
#Based on the correlation matrix ouput, we have decided to remove some highly correlated variables
high_corr_var = ['calculatedbathnbr', 'fullbathcnt', 'finishedsquarefeet12', 'taxamount', 'landtaxvaluedollarcnt', 'numberofstories', 'rawcensustractandblock', 'pooltypeid7']

try:
    final = final.drop(columns = high_corr_var)

except:
    pass

#visually plot correlation matrix after removal of highly correlated variables
num_cols = final.select_dtypes(include = ['int64', 'float64'])

In [18]:
#run correlation matrix on numeric columns again to visualize the matrix after removing high corr variables
plt.close('all')
correlation_matrix = num_cols.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()




IMPUTE MISSING VALUE

Lets check for null values again, now that we have filled NaN with zeroes, we need to decide what to do with the remaining NaN columns

In [19]:
final.isnull().sum()
null_percent2 = final.isnull().mean()*100
null_percent2[null_percent2>0]

buildingqualitytypeid    35.741493
heatingorsystemtypeid    36.565930
lotsizesquarefeet        10.738987
propertyzoningdesc       35.027314
unitcnt                  34.990352
dtype: float64

we will just drop property zoning desc as it is just a description field

In [20]:
final = final.drop(columns = 'propertyzoningdesc')

for the remaining 4 columns, we will try to predict these based on the remaining information we have

In [21]:
final.isnull().sum()
null_percent3 = final.isnull().mean()*100
null_percent3[null_percent3>0]

buildingqualitytypeid    35.741493
heatingorsystemtypeid    36.565930
lotsizesquarefeet        10.738987
unitcnt                  34.990352
dtype: float64

In [22]:
#what are the features that can be used to predict the above 4?

features = ['bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedfinishedsquarefeet', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'pooltypeid10', 'pooltypeid2',
       'propertylandusetypeid', 'roomcnt', 'unitcnt', 'yearbuilt', 'fireplaceflag',
       'taxdelinquencyflag']

final[features].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159624 entries, 0 to 167850
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   bathroomcnt                   159624 non-null  float64       
 1   bedroomcnt                    159624 non-null  float64       
 2   buildingqualitytypeid         102572 non-null  category      
 3   calculatedfinishedsquarefeet  159624 non-null  float64       
 4   hashottuborspa                159624 non-null  float64       
 5   heatingorsystemtypeid         101256 non-null  category      
 6   latitude                      159624 non-null  float64       
 7   longitude                     159624 non-null  float64       
 8   lotsizesquarefeet             142482 non-null  float64       
 9   poolcnt                       159624 non-null  float64       
 10  pooltypeid10                  159624 non-null  float64       
 11  pooltypeid2  

In [23]:
final_features = final[features]

final_encoded = pd.get_dummies(final_features, columns = ['propertylandusetypeid']) # Why doing this?

final_encoded



,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,...,propertylandusetypeid_248.0,propertylandusetypeid_260.0,propertylandusetypeid_261.0,propertylandusetypeid_263.0,propertylandusetypeid_264.0,propertylandusetypeid_265.0,propertylandusetypeid_266.0,propertylandusetypeid_267.0,propertylandusetypeid_269.0,propertylandusetypeid_275.0
0,2.5,3.0,NaN,1264.0,0.0,NaN,34303597.0,-119287236.0,1735.0,0.0,...,0,0,0,0,0,1,0,0,0,0
2,2.0,3.0,NaN,1101.0,0.0,NaN,34340801.0,-119079610.0,6569.0,0.0,...,0,0,1,0,0,0,0,0,0,0
3,1.5,2.0,NaN,1554.0,0.0,NaN,34354313.0,-119076405.0,7400.0,0.0,...,0,0,1,0,0,0,0,0,0,0
4,2.5,4.0,NaN,2415.0,0.0,NaN,34266578.0,-119165392.0,6326.0,0.0,...,0,0,1,0,0,0,0,0,0,0
5,2.5,4.0,NaN,2882.0,0.0,NaN,34240014.0,-119024793.0,10000.0,0.0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167846,2.5,3.0,NaN,1631.0,0.0,NaN,33595728.0,-117726062.0,NaN,0.0,...,0,0,0,0,0,0,1,0,0,0
167847,2.5,3.0,NaN,2658.0,0.0,NaN,33870884.0,-117823709.0,NaN,0.0,...,0,0,0,0,0,0,1,0,0,0
167848,1.0,3.0,5.0,792.0,0.0,7.0,34012294.0,-118246482.0,2704.0,0.0,...,0,0,1,0,0,0,0,0,0,0
167849,2.0,2.0,12.0,1320.0,0.0,2.0,34100060.0,-118344916.0,NaN,0.0,...,0,0,0,0,0,0,1,0,0,0


In [24]:
#takes about 10 minutes to run

impute_cols = ['buildingqualitytypeid',
                'heatingorsystemtypeid',
                'lotsizesquarefeet',
                'unitcnt']

to_impute = final[impute_cols]

knn_imputer = KNNImputer(n_neighbors = 5)

knn_imputer.fit(to_impute.dropna())

imputed = knn_imputer.transform(to_impute)

final[impute_cols] = imputed


In [25]:
#check out your new dataset, no more NULLS!

final.info()
final.columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159624 entries, 0 to 167850
Data columns (total 30 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   parcelid                      159624 non-null  category      
 1   bathroomcnt                   159624 non-null  float64       
 2   bedroomcnt                    159624 non-null  float64       
 3   buildingqualitytypeid         159624 non-null  float64       
 4   calculatedfinishedsquarefeet  159624 non-null  float64       
 5   fips                          159624 non-null  float64       
 6   hashottuborspa                159624 non-null  float64       
 7   heatingorsystemtypeid         159624 non-null  float64       
 8   latitude                      159624 non-null  float64       
 9   longitude                     159624 non-null  float64       
 10  lotsizesquarefeet             159624 non-null  float64       
 11  poolcnt      

Index(['parcelid', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedfinishedsquarefeet', 'fips', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'pooltypeid10', 'pooltypeid2', 'propertycountylandusecode',
       'propertylandusetypeid', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'unitcnt', 'yearbuilt', 'fireplaceflag',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear',
       'taxdelinquencyflag', 'censustractandblock', 'logerror',
       'transactiondate'],
      dtype='object')

OUTLIER ANALYSIS

All the values have been imputed, we will proceed with outlier analysis

In [26]:
subset_1 = final[['parcelid', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedfinishedsquarefeet', 'fips', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'pooltypeid10', 'pooltypeid2', 'propertycountylandusecode',
       ]]
first_5_rows = subset_1
print(first_5_rows.head(5))
first_5_rows.describe(include='all')

   parcelid  bathroomcnt  bedroomcnt  buildingqualitytypeid  \
0  17073783          2.5         3.0                    6.2   
2  17100444          2.0         3.0                    6.6   
3  17102429          1.5         2.0                    5.2   
4  17109604          2.5         4.0                    5.0   
5  17125829          2.5         4.0                    7.6   

   calculatedfinishedsquarefeet    fips  hashottuborspa  \
0                        1264.0  6111.0             0.0   
2                        1101.0  6111.0             0.0   
3                        1554.0  6111.0             0.0   
4                        2415.0  6111.0             0.0   
5                        2882.0  6111.0             0.0   

   heatingorsystemtypeid    latitude    longitude  lotsizesquarefeet  poolcnt  \
0                    2.0  34303597.0 -119287236.0             1735.0      0.0   
2                    2.0  34340801.0 -119079610.0             6569.0      0.0   
3                    4.

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,fips,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,pooltypeid10,pooltypeid2,propertycountylandusecode
count,159624.0,159624.000000,159624.000000,159624.000000,159624.000000,159624.000000,159624.0,159624.000000,1.596240e+05,1.596240e+05,1.596240e+05,159624.000000,159624.000000,159624.000000,159624
unique,159624.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68
top,10711738.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0100
freq,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55202
mean,NaN,2.305408,3.068937,6.050559,1766.782508,6048.819401,0.0,3.927658,3.400482e+07,-1.181998e+08,2.939636e+04,0.205051,0.009785,0.013450,NaN
std,NaN,0.972120,1.109379,1.578025,924.820853,20.710978,0.0,2.989338,2.627357e+05,3.580468e+05,1.134057e+05,0.403740,0.098437,0.115193,NaN
min,NaN,1.000000,0.000000,1.000000,2.000000,6037.000000,0.0,1.000000,3.333930e+07,-1.194753e+08,1.670000e+02,0.000000,0.000000,0.000000,NaN
25%,NaN,2.000000,2.000000,5.000000,1180.000000,6037.000000,0.0,2.000000,3.381352e+07,-1.184089e+08,5.951000e+03,0.000000,0.000000,0.000000,NaN
50%,NaN,2.000000,3.000000,6.024604,1535.000000,6037.000000,0.0,3.000000,3.401988e+07,-1.181754e+08,7.520000e+03,0.000000,0.000000,0.000000,NaN
75%,NaN,3.000000,4.000000,7.000000,2085.000000,6059.000000,0.0,6.000000,3.417129e+07,-1.179251e+08,1.799700e+04,0.000000,0.000000,0.000000,NaN


In [27]:
subset = final[['propertylandusetypeid', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'unitcnt', 'yearbuilt', 'fireplaceflag',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear',
       'taxdelinquencyflag', 'censustractandblock', 'logerror',
       'transactiondate']]
first_5_rows = subset
print(first_5_rows.head(5))
first_5_rows.describe(include='all')

  propertylandusetypeid regionidcity regionidcounty regionidzip  roomcnt  \
0                 265.0      34543.0         2061.0     97081.0      5.0   
2                 261.0      26965.0         2061.0     97113.0      5.0   
3                 261.0      26965.0         2061.0     97113.0      5.0   
4                 261.0      34543.0         2061.0     97084.0      8.0   
5                 261.0      51239.0         2061.0     97089.0      8.0   

   unitcnt                     yearbuilt  fireplaceflag  \
0      1.0 1970-01-01 00:00:00.000001986            0.0   
2      1.0 1970-01-01 00:00:00.000001956            0.0   
3      1.0 1970-01-01 00:00:00.000001965            0.0   
4      1.2 1970-01-01 00:00:00.000001984            0.0   
5      1.0 1970-01-01 00:00:00.000001980            0.0   

   structuretaxvaluedollarcnt  taxvaluedollarcnt  \
0                    115087.0           191811.0   
2                     33619.0            47853.0   
3                     45609.0   

C:\Users\rolan\AppData\Local\Temp\ipykernel_12280\4235403856.py:8: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  first_5_rows.describe(include='all')
C:\Users\rolan\AppData\Local\Temp\ipykernel_12280\4235403856.py:8: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  first_5_rows.describe(include='all')
C:\Users\rolan\AppData\Local\Temp\ipykernel_12280\4235403856.py:8: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the f

,propertylandusetypeid,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yearbuilt,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,taxdelinquencyflag,censustractandblock,logerror,transactiondate
count,159624.0,159624.0,159624.0,159624.0,159624.000000,159624.000000,159624,159624.0,1.596240e+05,1.596240e+05,159624,159624.000000,1.596240e+05,159624.000000,159624
unique,13.0,177.0,3.0,387.0,NaN,NaN,138,NaN,NaN,NaN,2,NaN,NaN,NaN,616
top,261.0,12447.0,3101.0,97319.0,NaN,NaN,1970-01-01 00:00:00.000001955,NaN,NaN,NaN,1970-01-01 00:00:00.000002015,NaN,NaN,NaN,2017-06-30 00:00:00
freq,108208.0,37457.0,104126.0,1701.0,NaN,NaN,4066,NaN,NaN,NaN,84628,NaN,NaN,NaN,1153
first,NaN,NaN,NaN,NaN,NaN,NaN,1970-01-01 00:00:00.000001824,NaN,NaN,NaN,1970-01-01 00:00:00.000002015,NaN,NaN,NaN,2016-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,1970-01-01 00:00:00.000002016,NaN,NaN,NaN,1970-01-01 00:00:00.000002016,NaN,NaN,NaN,2017-09-25 00:00:00
mean,NaN,NaN,NaN,NaN,1.508363,1.067787,NaN,0.0,1.825967e+05,4.703805e+05,NaN,0.027508,6.049393e+13,0.012806,NaN
std,NaN,NaN,NaN,NaN,2.843725,0.545411,NaN,0.0,2.142635e+05,5.983579e+05,NaN,0.163560,1.077382e+12,0.161677,NaN
min,NaN,NaN,NaN,NaN,0.000000,1.000000,NaN,0.0,4.400000e+01,3.254000e+03,NaN,0.000000,6.037101e+13,-4.655420,NaN
25%,NaN,NaN,NaN,NaN,0.000000,1.000000,NaN,0.0,8.271900e+04,2.030318e+05,NaN,0.000000,6.037310e+13,-0.025300,NaN


Transposed the 'yearbuilt' column (datetime) to 'age_of_house' & Removing columns that are irrelevant

In [28]:
current_year = dt.datetime.now().year

# Ensure 'yearbuilt' is in the correct format (if it's not already)
final['yearbuilt'] = pd.to_datetime(final['yearbuilt'])

# Creating the 'age_of_house' column
final['age_of_house'] = current_year - final['yearbuilt'].dt.year

# Remove the specified columns
columns_remove = ['parcelid', 'fips', 'heatingorsystemtypeid', 'pooltypeid10', 'pooltypeid2', 'propertylandusetypeid', 'fireplaceflag', 'assessmentyear', 'transactiondate', 'taxdelinquencyflag', 'yearbuilt']
final_relevant = final.drop(columns=columns_remove)

Finding Outliers

In [29]:
final_relevant.columns

Index(['bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedfinishedsquarefeet', 'hashottuborspa', 'latitude',
       'longitude', 'lotsizesquarefeet', 'poolcnt',
       'propertycountylandusecode', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'unitcnt', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'censustractandblock', 'logerror', 'age_of_house'],
      dtype='object')

In [30]:
# Select columns to create box plots
columns = ['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'latitude','longitude','lotsizesquarefeet', 'roomcnt', 'unitcnt','buildingqualitytypeid', 'age_of_house', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'logerror']

# Create subplots for each column
fig, axes = plt.subplots(1, len(columns), figsize=(25, 5))

# Generate box plots for each column
for i, col in enumerate(columns):
    axes[i].boxplot(final_relevant[col], vert=False)
    axes[i].set_title(col)

plt.show()


* I chose 13 variables above to test visual of the outliers
* as you can see above, lotsizesquarefeet, roomcnt, and unitcnt data are highly concentrated within a narrow range, and there are too many data points that fall far outside the range.
* This could indicate a strong central tendency, while also having a significant number of extreme values.

In [31]:
final_relevant = final_relevant.drop(columns=['roomcnt', 'unitcnt'])

* I'm removing 2 variables: roomcnt and unitcnt, because it looks insignificant
* Below, I want to check what values are the outliers for each variable

In [32]:
# matplotlib.rcParams['figure.figsize'] = [10.0,3.0]
# sns.boxplot(x = final_encoded['bathroomcnt'])

outliers_columns = ['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'latitude','longitude','lotsizesquarefeet','buildingqualitytypeid', 'age_of_house', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'logerror']
thresholds = {}

for column in outliers_columns:
    # Calculate the first and third quartiles (Q1 & Q3)
    Q1 = final_relevant[column].quantile(0.25)
    Q3 = final_relevant[column].quantile(0.75)
    # Calculate the Interquartile Range (IQR)
    IQR = Q3 - Q1
    # Define the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Store the thresholds in a dictionary
    thresholds[column] = (lower_bound, upper_bound)

print("Outlier Thresholds:")
for column, (lower_bound, upper_bound) in thresholds.items():
    print(f"{column}: < {lower_bound} or > {upper_bound}")

Outlier Thresholds:
bathroomcnt: < 0.5 or > 4.5
bedroomcnt: < -1.0 or > 7.0
calculatedfinishedsquarefeet: < -177.5 or > 3442.5
latitude: < 33276854.625 or > 34707951.625
longitude: < -119134545.5 or > -117199451.5
lotsizesquarefeet: < -12118.0 or > 36066.0
buildingqualitytypeid: < 2.0 or > 10.0
age_of_house: < 53.0 or > 53.0
structuretaxvaluedollarcnt: < -111500.25 or > 406417.75
taxvaluedollarcnt: < -319484.625 or > 1073892.375
logerror: < -0.12070000000000002 or > 0.1337


### Removing rows based on the Z-Score

In [33]:
outliers_columns = ['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'latitude','longitude','lotsizesquarefeet','buildingqualitytypeid', 'age_of_house', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'logerror']

# Create a mask to keep track of rows to remove
remove_rows_mask = np.zeros(final_relevant.shape[0], dtype=bool)

threshold = 3

for column in outliers_columns:
    z_scores = zscore(final_relevant[column])
    # Identify rows with z-scores outside the specified threshold
    outliers_mask = (z_scores > threshold) | (z_scores < -threshold)
    # Update the remove_rows_mask with the outliers from the current column
    remove_rows_mask |= outliers_mask

# Filter the DataFrame to remove rows with outliers
filtered_df = final_relevant[~remove_rows_mask]

# Display the dimensions of the filtered DataFrame
print("Original DataFrame Shape:", final_relevant.shape)
print("Filtered DataFrame Shape:", filtered_df.shape)
print("Unique Z-Scores:", np.unique(z_scores))

Original DataFrame Shape: (159624, 18)
Filtered DataFrame Shape: (147137, 18)
Unique Z-Scores: [-28.87389925 -28.56203929 -27.97444559 ...  27.87788221  29.22006993
  32.47347513]


### The looks after cleaning

In [34]:
print("Filtered DataFrame Shape:", filtered_df.shape)
filtered_df.describe(include='all')

Filtered DataFrame Shape: (147137, 18)


,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,propertycountylandusecode,regionidcity,regionidcounty,regionidzip,structuretaxvaluedollarcnt,taxvaluedollarcnt,censustractandblock,logerror,age_of_house
count,147137.000000,147137.000000,147137.000000,147137.000000,147137.0,1.471370e+05,1.471370e+05,147137.000000,147137.000000,147137,147137.0,147137.0,147137.0,147137.000000,1.471370e+05,1.471370e+05,147137.000000,147137.0
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57,175.0,3.0,385.0,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0100,12447.0,3101.0,97319.0,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52816,32978.0,94270.0,1649.0,NaN,NaN,NaN,NaN,NaN
mean,2.203423,3.004717,6.069453,1663.803068,0.0,3.400245e+07,-1.181884e+08,20954.679461,0.190374,NaN,NaN,NaN,NaN,156560.327056,4.036016e+05,6.049698e+13,0.007794,53.0
std,0.796852,0.980982,1.366652,693.715810,0.0,2.648336e+05,3.560894e+05,40780.563612,0.392597,NaN,NaN,NaN,NaN,112408.862691,3.042229e+05,1.120641e+12,0.085685,0.0
min,1.000000,0.000000,2.000000,2.000000,0.0,3.333930e+07,-1.192739e+08,167.000000,0.000000,NaN,NaN,NaN,NaN,44.000000,3.254000e+03,6.037101e+13,-0.471600,53.0
25%,2.000000,2.000000,5.000000,1171.000000,0.0,3.380812e+07,-1.183963e+08,5863.000000,0.000000,NaN,NaN,NaN,NaN,80635.000000,1.974950e+05,6.037400e+13,-0.025083,53.0
50%,2.000000,3.000000,6.024604,1506.000000,0.0,3.400956e+07,-1.181590e+08,7437.000000,0.000000,NaN,NaN,NaN,NaN,128818.000000,3.384200e+05,6.037603e+13,0.005000,53.0
75%,3.000000,4.000000,7.000000,2003.000000,0.0,3.417310e+07,-1.179165e+08,15238.000000,0.000000,NaN,NaN,NaN,NaN,199000.000000,5.200000e+05,6.059042e+13,0.035785,53.0


In [35]:

# Select columns to create box plots
columns = ['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'latitude','longitude','lotsizesquarefeet','buildingqualitytypeid', 'age_of_house', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'logerror']

# Create subplots for each column
fig, axes = plt.subplots(1, len(columns), figsize=(25, 5))

# Generate box plots for each column
for i, col in enumerate(columns):
    axes[i].boxplot(filtered_df[col], vert=False)
    axes[i].set_title(col)

plt.show()

## Modeling

### Linear Regression

In [36]:
filtered_df.dtypes

bathroomcnt                      float64
bedroomcnt                       float64
buildingqualitytypeid            float64
calculatedfinishedsquarefeet     float64
hashottuborspa                   float64
latitude                         float64
longitude                        float64
lotsizesquarefeet                float64
poolcnt                          float64
propertycountylandusecode         object
regionidcity                    category
regionidcounty                  category
regionidzip                     category
structuretaxvaluedollarcnt       float64
taxvaluedollarcnt                float64
censustractandblock              float64
logerror                         float64
age_of_house                       int64
dtype: object

In [ ]:
# Encode categorical columns
categorical_columns = ['propertycountylandusecode', 'regionidcity', 'regionidcounty', 'regionidzip']
filtered_df_encoded = pd.get_dummies(filtered_df, columns=categorical_columns)

In [37]:
# splitting the data and fitting the model
X_train, X_test, y_train, y_test = train_test_split(filtered_df_encoded.drop(columns=['taxvaluedollarcnt']), filtered_df_encoded['taxvaluedollarcnt'], shuffle=True, test_size=0.3)

model = LinearRegression()
model.fit(X_train, y_train)

y_predict_train = model.predict(X_train)
y_predict_test = model.predict(X_test)

# train_accuracy = accuracy_score(y_train, np.round(y_predict_train))
# test_accuracy = accuracy_score(y_test, np.round(y_predict_test))

# train_accuracy
# test_accuracy

# y_predict_train

#### Error Rate

In [38]:
mean_absolute_error(y_test, np.round(y_predict_test))

121387.17824294324

In [39]:
error = mean_absolute_error(y_test, np.round(y_predict_test)) / np.mean(y_test)
error

0.3018155365182427

### ARIMA

In [42]:
X = filtered_df_encoded.drop(columns=['taxvaluedollarcnt'])
y = filtered_df_encoded['taxvaluedollarcnt']
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.3)

# Create and fit the ARIMA model
order = (1, 1, 1)  
model = ARIMA(y_train, order=order)
fitted_model = model.fit()

# Make predictions
predictions = fitted_model.forecast(steps=len(y_test))

c:\Users\rolan\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\rolan\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\rolan\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\rolan\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\rolan\AppData\Local

#### Error Rate

In [44]:
mean_absolute_error(y_test, np.round(predictions))

220273.36552489692

In [45]:
error = mean_absolute_error(y_test, np.round(predictions)) / np.mean(y_test)
error

0.5445275177952342

### Decision Tree

In [47]:
X_train, X_test, y_train, y_test = train_test_split(filtered_df_encoded.drop(columns=['taxvaluedollarcnt']), filtered_df_encoded['taxvaluedollarcnt'], shuffle=True, test_size=0.3)

model = DecisionTreeRegressor()
model.fit(X_train, y_train)

y_predict_train = model.predict(X_train)
y_predict_test = model.predict(X_test)

#### Error Rate

In [48]:
mean_absolute_error(y_test, np.round(y_predict_test))

126542.188165466

In [49]:
error = mean_absolute_error(y_test, np.round(y_predict_test)) / np.mean(y_test)
error

0.3140629405267087

### Model Conclusion

##### Error Rate:
* Linear Regression: 30.18%
* ARIMA: 54.45%
* Decision Tree: 31.40%